In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
df = pd.read_csv("AugmentedExercises.csv")
df.head()

,Title,Desc,Type,BodyPart,Equipment,Level,Rating,Movement,MovementType,Intensity,InjuryRisk,SecondaryMuscles
0,Barbell roll-out,The barbell roll-out is an abdominal exercise ...,Strength,Abdominals,Barbell,Intermediate,8.9,core,Isolation,High,Low,NaN
1,Barbell Ab Rollout - On Knees,The barbell roll-out is an abdominal exercise ...,Strength,Abdominals,Barbell,Intermediate,8.9,core,Isolation,High,Low,NaN
2,Dumbbell V-Sit Cross Jab,The dumbbell V-sit cross jab is a hybrid movem...,Strength,Abdominals,Dumbbell,Intermediate,9.3,core,Isolation,Medium,Low,NaN
3,Dumbbell spell caster,The dumbbell spell caster is an exercise that ...,Strength,Abdominals,Dumbbell,Beginner,9.3,core,Isolation,Medium,Low,Obliques
4,Landmine twist,The landmine twist is a rotational abdominal m...,Strength,Abdominals,Other,Intermediate,9.5,core,Isolation,Low,Low,Obliques


In [3]:
# Split SecondaryMuscles into individual terms
df['SecondaryMuscles'] = df['SecondaryMuscles'].fillna('').str.split(', ')

# Create binary matrices for categorical features
bodypart_dummies = pd.get_dummies(df['BodyPart'], prefix='BodyPart')
movement_dummies = pd.get_dummies(df['Movement'], prefix='Movement')
equipment_dummies = pd.get_dummies(df['Equipment'], prefix='Equipment')
level_dummies = pd.get_dummies(df['Level'], prefix='Level')

# Explode and encode SecondaryMuscles
secondary_exp = df['SecondaryMuscles'].explode()
secondary_dummies = pd.get_dummies(secondary_exp, prefix='Secondary').groupby(level=0).max()

# Combine all categorical features
categorical_matrix = pd.concat([
    bodypart_dummies, 
    movement_dummies, 
    equipment_dummies,
    level_dummies,
    secondary_dummies
], axis=1).fillna(0)

In [4]:
categorical_sim = cosine_similarity(categorical_matrix)

In [5]:
tfidf = TfidfVectorizer(stop_words='english')
desc_matrix = tfidf.fit_transform(df['Desc'])
text_sim = linear_kernel(desc_matrix, desc_matrix)
combined_sim = 0.45 * categorical_sim + 0.55 * text_sim

In [6]:
indices = pd.Series(df.index, index=df['Title']).drop_duplicates()

In [7]:
def get_recommendations(title, similarity_matrix=combined_sim, num_recommend=10):
    idx = indices[title]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in sim_scores[1:num_recommend+1]]
    return df.iloc[top_indices]

In [8]:
get_recommendations('Pushups', num_recommend = 10)

,Title,Desc,Type,BodyPart,Equipment,Level,Rating,Movement,MovementType,Intensity,InjuryRisk,SecondaryMuscles
75,Chest dip,The chest dip is a bodyweight exercise perform...,Strength,Chest,Other,Intermediate,9.0,push,Isolation,Low,Low,"[Triceps, Shoulders]"
74,Close-grip EZ-bar bench press,The close-grip EZ-bar bench press is an exerci...,Strength,Chest,E-Z Curl Bar,Intermediate,8.8,push,Isolation,Low,Low,"[Triceps, Shoulders]"
61,Hands-elevated push-up,The hands-elevated push-up is a variation on t...,Strength,Chest,Body Only,Beginner,8.7,push,Isolation,Low,Low,[Triceps]
54,Close-grip bench press,The close-grip bench press is a popular exerci...,Strength,Chest,Barbell,Intermediate,9.1,push,Isolation,High,Low,[Triceps]
67,Dumbbell Flyes,The dumbbell chest fly is a popular exercise t...,Strength,Chest,Dumbbell,Intermediate,9.1,push,Isolation,Medium,Low,[Shoulders]
72,Neutral-grip dumbbell bench press,The neutral-grip dumbbell bench press is a var...,Strength,Chest,Dumbbell,Intermediate,8.7,push,Isolation,Medium,Low,[Triceps]
60,Incline Push-Up,The hands-elevated push-up is a variation on t...,Strength,Chest,Body Only,Intermediate,8.8,push,Isolation,Low,Low,[]
102,Pullups,The pull-up is a multi-joint bodyweight exerci...,Strength,Lats,Body Only,Intermediate,9.2,pull,Isolation,Low,Low,[]
55,Barbell Bench Press - Medium Grip,The bench press is a compound exercise that bu...,Strength,Chest,Barbell,Intermediate,9.0,push,Compound,High,Low,[Triceps]
201,Push-Ups - Close Triceps Position,The close push-up is an upper-body exercise th...,Strength,Triceps,Body Only,Intermediate,9.0,push,Isolation,Low,Low,[]


In [24]:
combined_sim

array([[1.        , 1.        , 0.37362764, ..., 0.18973729, 0.18912179,
        0.18564702],
       [1.        , 1.        , 0.37362764, ..., 0.18973729, 0.18912179,
        0.18564702],
       [0.37362764, 0.37362764, 1.        , ..., 0.20015696, 0.20441222,
        0.18837199],
       ...,
       [0.18973729, 0.18973729, 0.20015696, ..., 1.        , 0.42214091,
        0.37455877],
       [0.18912179, 0.18912179, 0.20441222, ..., 0.42214091, 1.        ,
        0.65123879],
       [0.18564702, 0.18564702, 0.18837199, ..., 0.37455877, 0.65123879,
        1.        ]])